# Exercícios

1. Altere o programa abaixo para que identifique apenas as palavras que começam com a, tenham qualquer coisa no meio, e terminem com b.

In [ ]:
import re
def text_match(text):
        patterns = '^a.*b$'
        if re.search(patterns,  text):
                return 'Encontrei!'
        else:
                return('Oxe!')

print(text_match("bb"))
print(text_match("a1b2b"))
print(text_match("abcde"))
print(text_match("abcdeb"))

2. Altere a variavel patterns do código da questão 1 e escreva um programa em Python que identifique números de telefone no formato 
 * (71) 98765-4321

In [ ]:
import re
def text_match(text):
        patterns = '\([0-9]{2}\)[0-9]{5}-[0-9]{4}'
        if re.search(patterns,  text):
                return 'Encontrei!'
        else:
                return('Oxe!')

print(text_match("(71)982765-43221"))
print(text_match("982765-43221"))
print(text_match("(71)98255-4321"))

3. Altere a variavel patterns do código da questão 1 e escreva um programa em Python que identifique CPFs no formato 
 * 123.456.789-10

In [ ]:
import re
def text_match(text):
        patterns = '^([0-9]{3}.)([0-9]{3}.)[0-9]{3}-[0-9]{2}'
        if re.search(patterns,  text):
                return 'Encontrei!'
        else:
                return('Oxe!')

print(text_match("123.456.789-10"))
print(text_match("111123.456.789-10000"))
print(text_match("12345678910"))

4. Altere a variavel patterns do código da questão 1 e escreva um programa em Python que identifique palavras em que apenas a primeira letra é maiúscula.

In [ ]:
import re
def text_match(text):
        patterns = '^[A-Z].*'
        if re.search(patterns,  text):
                return 'Encontrei!'
        else:
                return('Oxe!')

print(text_match("Baaa"))
print(text_match("aaa"))
print(text_match("aaBBa"))

Encontrei!
Oxe!
Oxe!


# Classificando Tweets sobre eleições com Expressões Regulares

* Objetivos
  * Apresentação da biblioteca re da linguagem Python
  * Classificar Tweets automaticamente utilizando expressões regulares 
  * Identificar os políticos mais comentados no Twitter

##1  - Utilize o seu BEARER_TOKEN para realizar as consultas na API do Twitter.

In [ ]:
# Use o seu próprio BEARER_TOKEN
bearer_token = ''

## 2 - Defina a sua chave de busca para tweets no Tweeter. Escolha os políticos (presidente da República, governador, senador, deputado federal, deputado estadual ou deputado distrital) que você quer monitorar.

In [ ]:
# Essa chave de consulta ficará em branco
query = 'lula OR bolsonaro'

In [ ]:
#@title Mantenha esse código oculto
from tqdm import tqdm
import json
import csv
import requests
import pandas
import time
from requests.utils import quote
# import emoji
import re
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def connect_to_endpoint(url, headers):
    """ Connects to API URL endpoint """
    for i in range(1, 14):
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                return response.json()
        except Exception as e:
      
            print("Connection failed, retrying...",i)
            time.sleep(5)

def json_extract(obj, key):
    """Recursively fetch values from nested JSON."""
    arr = []

    def extract(obj, arr, key):
        """Recursively search for values of key in JSON tree."""
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr, key)
                elif k == key:
                    arr.append(v)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    values = extract(obj, arr, key)
    return values

def main(token):
    stop = False
    tweets = set()
    # for i in tqdm(range(10)):
    for i in range(10):
        print()
        headers = {"Authorization": "Bearer {}".format(bearer_token)}
        query_field = quote(query, safe='')
        
        if token == '':
          url = f'https://api.twitter.com/2/tweets/search/recent?query={query_field}'
        else:
          url = f'https://api.twitter.com/2/tweets/search/recent?query={query_field}'+'&next_token='+token

        time.sleep(3.2) # Sleep for 3 seconds
        try:
            data = connect_to_endpoint(url, headers)
        except Exception as e:
            print("Oh no!!!")
        id = json_extract(data, 'id')
        created_at = json_extract(data, 'id')
        author_id = json_extract(data, 'author_id')
        text = json_extract(data, 'text')

        
        for t in text:
            tweets.add(t)       
        
        try:
            token = json_extract(data, 'next_token')[0]
            with open(token+'.json', 'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=4, sort_keys=True)
            
            break
        except:
            token = ''
            with open('last.json', 'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=4, sort_keys=True)
            print("finished")
            stop = True

        if stop:
            break
    
    complete = list(tweets)
    return(complete,token)
    


## 3 - Sua tarefa: Defina uma expressão regular que seja capaz de identificar se um tweet é contra ou a favor do seu candidato do coração nestas eleições de 2022. Caso necessário, modifique o gráfico de pizza para melhor visualização.

A entrega desta atividade requer dois arquivos:
* O arquivo fonte do Google Colab na extensão ipynb
* A imagem do gráfico de pizza gerada após a execução do seu código

In [ ]:
from IPython.display import display, clear_output

if __name__ == '__main__':
  
  # Faz uma coleta inicial de Tweets
  token = ''

  # Geracao de Graficos
  y = np.array([0, 0, 0])
  labels = ["Lula", "Bolsonaro", "Ambos"]

  # Realiza 5 buscas no Twitter, altere essa quantidade como desejar
  buscas = 5
  for i in range(buscas):
    # Utiliza token para continuar buscando Tweets do último ponto de retorno
    tweets,token = main(token)  
    for t in tweets:
      plt.pause(0.5)
      plt.clf() 
      clear_output(wait=True)
      print(t)

      # Crie suas expressões regulares aqui!!!
      favoravel = re.search(".*lula.*",t,re.IGNORECASE)
      contrario = re.search(".*bolsonaro.*",t,re.IGNORECASE)
      indefinido = re.search("(lula|bolsonaro)",t,re.IGNORECASE)

      if favoravel and contrario==None:
        y[0] = y[0]+1
      elif contrario and favoravel==None:
        y[1] = y[1]+1
      else:
        y[2] = y[2]+1
      
      plt.pie(y, labels = labels,autopct='%1.1f%%')
      plt.show()

    if token == '':
      break
    
